# Práctica 1 - Naive Bayes

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García

## Setup

### Importaciones

In [79]:
from Clasificador import Clasificador, ClasificadorNaiveBayes, ClasificadorNaiveBayesScikit
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB

import pandas as pd
from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB

### Estrategia de particionado

In [80]:
validacion_cruzada = ValidacionCruzada(numeroParticiones=5)
validacion_simple = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)

### Clasificador generico

In [81]:
clasificador = Clasificador()

### Utilerías

In [82]:
def validacion_naive_bayes_propio(datos, estrategia_particionado, con_laplace):
    return clasificador.validacion(estrategia_particionado, datos, ClasificadorNaiveBayes(con_laplace=con_laplace))

def validacion_naive_bayes(dataset):
    datos = Datos(f"{dataset}.csv")
    
    resultados_vc = validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=False)
    resultados_vc_cl = validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=True)
    
    resultados_vs = validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=False)
    resultados_vs_cl = validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=True)

    columnas = [
        "Conjunto de datos",
        "Estrategia Particionado",
        "Error Promedio sin CL",
        "Desviación estándar sin CL",
        "Error Promedio con CL",
        "Desviación estándar con CL",
    ]

    filas = [
        (
            dataset,
            "Validación Cruzada",
            resultados_vc[0],
            resultados_vc[1],
            resultados_vc_cl[0],
            resultados_vc_cl[1],
        ),
        (
            dataset,
            "Validación Simple",
            resultados_vs[0],
            resultados_vs[1],
            resultados_vs_cl[0],
            resultados_vs_cl[1],
        )
    ]

    return pd.DataFrame(filas, columns=columnas)

## Apartado 1 - Naive Bayes Propio

### Entrenamiento y clasificación para dataset __heart__

In [83]:
datos_heart = Datos("heart.csv")

In [84]:
heart_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", heart_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.1461095272036113
Desviación estándar:  0.05351009204304014


In [85]:
heart_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", heart_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.1461095272036113
Desviación estándar:  0.053510092043040136


In [86]:
heart_validacion_simple = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", heart_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.14036363636363638
Desviación estándar:  0.019048437605394635


In [87]:
heart_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_heart, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", heart_validacion_simple_laplace)

Validación simple con corrección de Laplace
Error promedio:  0.1410909090909091
Desviación estándar:  0.01970358863148231


### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [88]:
datos_tic_tac_toe = Datos("tic-tac-toe.csv")

In [89]:
tic_tac_toe_validacion_cruzada = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación cruzada sin corrección de Laplace", tic_tac_toe_validacion_cruzada)

Validación cruzada sin corrección de Laplace
Error promedio:  0.13626679636464206
Desviación estándar:  0.2732406767585417


In [90]:
tic_tac_toe_validacion_cruzada_laplace = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación cruzada con corrección de Laplace", tic_tac_toe_validacion_cruzada_laplace)

Validación cruzada con corrección de Laplace
Error promedio:  0.13937287214259148
Desviación estándar:  0.27952341003988734


In [91]:
tic_tac_toe_validacion_simple = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=False)
)

print_resultados("Validación simple sin corrección de Laplace", tic_tac_toe_validacion_simple)

Validación simple sin corrección de Laplace
Error promedio:  0.2738633723999577
Desviación estándar:  0.035642176772520295


In [92]:
tic_tac_toe_validacion_simple_laplace = clasificador.validacion(
    validacion_simple, datos_tic_tac_toe, ClasificadorNaiveBayes(con_laplace=True)
)

print_resultados("Validación simple con corrección de Laplace", tic_tac_toe_validacion_simple_laplace)

Validación simple con corrección de Laplace
Error promedio:  0.2817041495090276
Desviación estándar:  0.03606408078454856


### Analisis de resultados

In [93]:
resultados_naive_bayes_propio = pd.DataFrame(
    [
        (
            "tic-tac-toe",
            tic_tac_toe_validacion_cruzada[0],
            tic_tac_toe_validacion_cruzada[1],
            tic_tac_toe_validacion_cruzada_laplace[0],
            tic_tac_toe_validacion_cruzada_laplace[1],
            tic_tac_toe_validacion_simple[0],
            tic_tac_toe_validacion_simple[1],
            tic_tac_toe_validacion_simple_laplace[0],
            tic_tac_toe_validacion_simple_laplace[1],
        )
    ],
    columns=[
        "Dataset",
        "Promedio error validacion cruzada",
        "Desviación estandar error validación cruzada",
        "Promedio error validacion cruzada con Laplace",
        "Desviacion estandar error validacion cruzada con laplace",
        "Promedio error validacion simple",
        "Desviacion estandar error validacion simple",
        "Promedio error validacion simple con laplace",
        "Desviacion estandar error validacion simple con laplace",
    ],
)

In [94]:
resultados_naive_bayes_propio = pd.concat([resultados_heart, resultados_tic_tac_toe])
resultados_naive_bayes_propio

Dataset  Promedio error validacion cruzada  \
0  tic-tac-toe                           0.136267   

   Desviación estandar error validación cruzada  \
0                                      0.273241   

   Promedio error validacion cruzada con Laplace  \
0                                       0.139373   

   Desviacion estandar error validacion cruzada con laplace  \
0                                           0.279523          

   Promedio error validacion simple  \
0                          0.273863   

   Desviacion estandar error validacion simple  \
0                                     0.035642   

   Promedio error validacion simple con laplace  \
0                                      0.281704   

   Desviacion estandar error validacion simple con laplace  
0                                           0.036064

El error promedio en todos los casos es menor al 30%, lo cual consideramos un rendimiento regular. Lo interesante de estos datos es la reducción de la desviación estándar cuando se usa validación simple como estrategia de particionado. En conjuntos de datos pequeños, la validación cruzada puede tener una desviación estándar más alta debido a la limitada cantidad de datos para realizar las divisiones. La validación simple puede ser más estable en tales casos.

#### Efectos de Corrección de Laplace

Podemos ver que no hay mucho impacto de la corrección de Laplace en estos escenarios. Incluso, en la mayoría de escenarios, resulta en un error más grande. Esto puede ocurrir por la ausencia o cantidad reducida de atributos nominales donde se aplique esta corrección.

## Apartado 2 - Naive Bayes Scikit-learn

# Naive Bayes con Scikit

### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [98]:
tic_tac_toe_validacion_multinomial = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(MultinomialNB, False)
)

print_resultados("Validación con MultinomialNB sin oneHotEncoder", tic_tac_toe_validacion_multinomial)

AttributeError: 'DataFrame' object has no attribute '_validate_params'

In [ ]:
tic_tac_toe_validacion_multinomial_encoder = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(MultinomialNB, True)
)

print_resultados("Validación con MultinomialNB con oneHotEncoder", tic_tac_toe_validacion_multinomial_encoder)

In [ ]:
tic_tac_toe_validacion_gaussian = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(GaussianNB, False)
)

print_resultados("Validación con GaussianNB sin oneHotEncoder", tic_tac_toe_validacion_gaussian)

In [ ]:
tic_tac_toe_validacion_gaussian_encoder = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(GaussianNB, True)
)

print_resultados("Validación con MultinomialNB con oneHotEncoder", tic_tac_toe_validacion_gaussian_encoder)

In [ ]:
tic_tac_toe_validacion_categorico = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(CategoricalNB, False)
)

print_resultados("Validación con CategoricalNB sin oneHotEncoder", tic_tac_toe_validacion_categorico)

In [ ]:
tic_tac_toe_validacion_categorico_encoder = clasificador.validacion(
    validacion_cruzada, datos_tic_tac_toe, ClasificadorNaiveBayesScikit(CategoricalNB, True)
)

print_resultados("Validación con CategoricalNB con oneHotEncoder", tic_tac_toe_validacion_categorico_encoder)

### Análisis de resultados

In [100]:
resultados_naive_bayes_sk_ttt = pd.DataFrame(
    [
        (
            "tic-tac-toe",
            tic_tac_toe_validacion_multinomial[0],
            tic_tac_toe_validacion_multinomial[1],
            tic_tac_toe_validacion_gaussian[0],
            tic_tac_toe_validacion_gaussian[1],
            tic_tac_toe_validacion_categorico[0],
            tic_tac_toe_validacion_categorico[1],
        ),
        (
            "tic-tac-toe encoded",
            tic_tac_toe_validacion_multinomial_encoder[0],
            tic_tac_toe_validacion_multinomial_encoder[1],
            tic_tac_toe_validacion_gaussian_encoder[0],
            tic_tac_toe_validacion_gaussian_encoder[1],
            tic_tac_toe_validacion_categorico_encoder[0],
            tic_tac_toe_validacion_categorico_encoder[1],
        )
    ],
    columns=[
        "Dataset",
        "Promedio error clasificacion multinominal",
        "Desviación estandar error clasificacion multinominal",
        "Promedio error clasificacion gaussiana",
        "Desviacion estandar error clasificacion gaussiana",
        "Promedio error clasificacion categorica",
        "Desviacion estandar error clasificacion categorica",
    ],
)
resultados_naive_bayes_sk_ttt

TypeError: 'ClasificadorNaiveBayesScikit' object is not subscriptable

analisis...

### Entrenamiento y clasificación para dataset __heart__

In [ ]:
heart_validacion = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayesScikit(GaussianNB, False)
)

print_resultados("Validación de heart sin oneHotEncoder", heart_validacion)

In [ ]:
heart_validacion_encoder = clasificador.validacion(
    validacion_cruzada, datos_heart, ClasificadorNaiveBayesScikit(GaussianNB, True)
)

print_resultados("Validación de heart con oneHotEncoder", heart_validacion_encoder)

### Análisis de resultados

In [ ]:
resultados_naive_bayes_sk_heart = pd.DataFrame(
    [
        (
            "heart",
            heart_validacion[0],
            heart_validacion[1],
        ),
        (
            "heart encoded",
            heart_validacion_encoder[0],
            heart_validacion_encoder[1],
        )
    ],
    columns=[
        "Dataset",
        "Promedio error clasificacion gaussiana",
        "Desviacion estandar error clasificacion gaussiana",
    ],
)
resultados_naive_bayes_sk_heart

*Por qué crees que no utilizamos los otros dos algoritmos aquí?
¿qué transformación/es podríamos hacer para resolver el problema?*
Porque este dataset contiene atributos continuos. Para resolver el problema se pueden escalar estos atributos por ejemplo con la clase `StandardScaler` de sklearn. Así se transforma el atributo de tipo continuo a un tipo multinominal